In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('immobiliers.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1649 entries, 0 to 1648
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Annonce                   1649 non-null   object 
 1   Localisation              1649 non-null   object 
 2   Superficie                1649 non-null   object 
 3   Type de bien              197 non-null    object 
 4   Type de transaction       1649 non-null   object 
 5   Prix                      1430 non-null   float64
 6   Configuration des pièces  567 non-null    object 
 7   Salles de bains           1649 non-null   object 
 8   Chambres                  1649 non-null   object 
dtypes: float64(1), object(8)
memory usage: 116.1+ KB


In [4]:
# on s'interesse que sur les biens à vendre
# execute right after reading the csv, marra wahda barka
df = df[df['Type de transaction'] == 'À Vendre']
df = df.drop(columns=['Type de transaction'])
print(df.tail())

                                           Annonce Localisation Superficie  \
1644   Appartement (C2-8) en S+2 au deuxième étage         Sfax      False   
1645   Appartement (C2-9) en S+2 au deuxième étage         Sfax      125.8   
1646  Appartement (A4-9) en S+3 au quatrième étage         Sfax     183.21   
1647   Appartement (C2-3) en S+3 au deuxième étage         Sfax     202.79   
1648   Appartement (C2-6) en S+3 au deuxième étage         Sfax     191.02   

     Type de bien      Prix Configuration des pièces Salles de bains Chambres  
1644  Appartement  286269.0                      S+2           False    False  
1645  Appartement  232730.0                      S+2               1        2  
1646  Appartement  338939.0                      S+3               3        3  
1647  Appartement  375162.0                      S+3               3        3  
1648  Appartement  353387.0                      S+3               3        3  


In [8]:
condition1 = df[(df['Configuration des pièces'].notna()) & (df['Chambres']=='False')]
# print(condition1.shape)
# 12 row où nb chambres n'est affiché que dans l'annonce

# Extraire le nombre de pièces de 'Configuration des pièces' et l'attribuer à 'Chambres'
for idx, row in condition1.iterrows():
    config_pieces = row['Configuration des pièces']
    match = re.search(r'S\+(\d+)', config_pieces)
    if match:
        df.at[idx, 'Chambres'] = int(match.group(1))

df = df.drop(columns=['Configuration des pièces']) # on n'en a plus besoin
df.tail()

,Annonce,Localisation,Superficie,Type de bien,Prix,Salles de bains,Chambres
1644,Appartement (C2-8) en S+2 au deuxième étage,Sfax,False,Appartement,286269.0,False,2
1645,Appartement (C2-9) en S+2 au deuxième étage,Sfax,125.8,Appartement,232730.0,1,2
1646,Appartement (A4-9) en S+3 au quatrième étage,Sfax,183.21,Appartement,338939.0,3,3
1647,Appartement (C2-3) en S+3 au deuxième étage,Sfax,202.79,Appartement,375162.0,3,3
1648,Appartement (C2-6) en S+3 au deuxième étage,Sfax,191.02,Appartement,353387.0,3,3


In [6]:
df = df.drop_duplicates()

In [9]:
# Remplacer toutes les valeurs 'False' dans la colonne "Salles de bains" par la valeur par défaut 1
df['Salles de bains'].replace('False', 1, inplace=True)
df.tail()

C:\Users\My_Computing\AppData\Local\Temp\ipykernel_9472\3509720669.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Salles de bains'].replace('False', 1, inplace=True)


,Annonce,Localisation,Superficie,Type de bien,Prix,Salles de bains,Chambres
1644,Appartement (C2-8) en S+2 au deuxième étage,Sfax,False,Appartement,286269.0,1,2
1645,Appartement (C2-9) en S+2 au deuxième étage,Sfax,125.8,Appartement,232730.0,1,2
1646,Appartement (A4-9) en S+3 au quatrième étage,Sfax,183.21,Appartement,338939.0,3,3
1647,Appartement (C2-3) en S+3 au deuxième étage,Sfax,202.79,Appartement,375162.0,3,3
1648,Appartement (C2-6) en S+3 au deuxième étage,Sfax,191.02,Appartement,353387.0,3,3


In [10]:
df['Prix'] = pd.to_numeric(df['Prix'], errors='coerce') # convertir en numérique

In [11]:
# imputer les vm en utilisant la médiane des biens similaires (pour éviter les outliers)
# df['Prix'] = df.groupby(['Localisation', 'Chambres'])['Prix'].transform(lambda x: x.fillna(x.median())) 